In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Membuat dataset
data = {
    'GPA': [3.5, 3.0, 3.8, 2.8, 3.7, 3.2],
    'Skor_Tes': [85, 80, 90, 75, 88, 78],
    'Rekomendasi': ['Ya', 'Tidak', 'Ya', 'Tidak', 'Ya', 'Tidak'],
    'Diterima': ['Ya', 'Tidak', 'Ya', 'Tidak', 'Ya', 'Tidak']
}

df = pd.DataFrame(data)

# Encode data kategorikal
df['Rekomendasi'] = df['Rekomendasi'].map({'Ya': 1, 'Tidak': 0})
df['Diterima'] = df['Diterima'].map({'Ya': 1, 'Tidak': 0})

df


,GPA,Skor_Tes,Rekomendasi,Diterima
0,3.5,85,1,1
1,3.0,80,0,0
2,3.8,90,1,1
3,2.8,75,0,0
4,3.7,88,1,1
5,3.2,78,0,0


In [8]:
class ANNClassifier:
    def __init__(self, input_dim):
        self.model = Sequential()
        self.model.add(Dense(12, input_dim=input_dim, activation='relu'))
        self.model.add(Dense(8, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    def train(self, X_train, y_train, epochs=100, batch_size=10):
        # Menggunakan callback untuk mencetak loss dan akurasi setiap epoch
        class PrintEpochCallback(tf.keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs=None):
                print(f"Epoch {epoch + 1}: loss = {logs['loss']:.4f}, accuracy = {logs['accuracy']:.4f}")

        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[PrintEpochCallback()])

    def predict(self, X_test):
        return (self.model.predict(X_test) > 0.5).astype("int32")

    def accuracy(self, y_test, y_pred):
        return accuracy_score(y_test, y_pred)


In [9]:
# Membagi data menjadi fitur dan target
X = df[['GPA', 'Skor_Tes', 'Rekomendasi']]
y = df['Diterima']

# Membagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Standarisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Membuat instance dari ANNClassifier
ann_classifier = ANNClassifier(input_dim=X_train.shape[1])

# Melatih model dengan 100 epoch
ann_classifier.train(X_train, y_train, epochs=100)

# Membuat prediksi
y_pred = ann_classifier.predict(X_test)

# Menghitung akurasi
accuracy = ann_classifier.accuracy(y_test, y_pred)
print(f'Akurasi: {accuracy * 100:.2f}%')

# Menampilkan prediksi
predictions = pd.DataFrame({'Aktual': y_test, 'Prediksi': y_pred.flatten()})
print(predictions)


Epoch 1: loss = 0.6925, accuracy = 0.7500
Epoch 2: loss = 0.6880, accuracy = 0.7500
Epoch 3: loss = 0.6835, accuracy = 0.7500
Epoch 4: loss = 0.6792, accuracy = 0.7500
Epoch 5: loss = 0.6757, accuracy = 0.7500
Epoch 6: loss = 0.6721, accuracy = 0.7500
Epoch 7: loss = 0.6685, accuracy = 1.0000
Epoch 8: loss = 0.6649, accuracy = 1.0000
Epoch 9: loss = 0.6614, accuracy = 1.0000
Epoch 10: loss = 0.6578, accuracy = 1.0000
Epoch 11: loss = 0.6542, accuracy = 1.0000
Epoch 12: loss = 0.6505, accuracy = 1.0000
Epoch 13: loss = 0.6467, accuracy = 1.0000
Epoch 14: loss = 0.6426, accuracy = 1.0000
Epoch 15: loss = 0.6382, accuracy = 1.0000
Epoch 16: loss = 0.6338, accuracy = 1.0000
Epoch 17: loss = 0.6296, accuracy = 1.0000
Epoch 18: loss = 0.6255, accuracy = 1.0000
Epoch 19: loss = 0.6215, accuracy = 1.0000
Epoch 20: loss = 0.6174, accuracy = 1.0000
Epoch 21: loss = 0.6134, accuracy = 1.0000
Epoch 22: loss = 0.6090, accuracy = 1.0000
Epoch 23: loss = 0.6042, accuracy = 1.0000
Epoch 24: loss = 0.5

In [10]:
# Membuat prediksi untuk data baru
data_baru = pd.DataFrame({'GPA': [3.6], 'Skor_Tes': [82], 'Rekomendasi': [1]})
data_baru = scaler.transform(data_baru)
prediksi_baru = ann_classifier.predict(data_baru)
print(f'Prediksi untuk data baru (GPA=3.6, Skor Tes=82, Rekomendasi=Ya): {"Diterima" if prediksi_baru[0][0] == 1 else "Tidak Diterima"}')


1/1 [==============================] - 0s 23ms/step
Prediksi untuk data baru (GPA=3.6, Skor Tes=82, Rekomendasi=Ya): Diterima
